# OLS Regression on Rental Data and Geospacial Data

In [21]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures, StandardScaler


In [22]:
# Load the CSV file into a Pandas DataFrame
data1 = pd.read_csv('4_Final_Data.csv')

# Display the DataFrame
data1


,Latitude,Longitude,Monthly rent,Aconto,Square meters,Rooms,Furnished,Shareable,Pets allowed,Elevator,...,Floor_-1 to 0,Floor_1-3,Floor_3-8,Floor_9 or above,Distance to Transport Station (km),Distance to Beach (km),Distance to School (km),Distance to Restaurant (km),Distance to Hospital (km),Distance to Mall (km)
0,55.393856,10.394185,5400.0,800.0,62.0,2.0,0.0,0.0,0.0,1.0,...,0,1,0,0,0.227327,10.937984,0.347042,0.114848,1.856833,0.972492
1,56.300211,10.477949,7100.0,700.0,100.0,4.0,0.0,0.0,1.0,0.0,...,0,1,0,0,6.490199,5.187801,0.198759,0.081711,22.428327,20.441833
2,57.057390,9.936155,5800.0,300.0,64.0,2.0,0.0,0.0,0.0,0.0,...,0,1,0,0,2.057982,46.063274,0.894998,1.091345,1.726659,1.252278
3,55.721333,12.556324,8200.0,500.0,43.0,1.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.684702,2.087369,0.121870,0.253656,0.873095,1.532133
4,56.199538,10.198303,8195.0,600.0,60.0,3.0,0.0,1.0,1.0,1.0,...,0,1,0,0,1.569096,12.856874,0.929082,0.441412,1.803299,0.677731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9962,57.053293,9.911053,7600.0,850.0,73.0,2.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.158809,45.438610,0.556684,0.239033,0.530896,0.832760
9963,55.485419,9.500225,8250.0,1000.0,84.0,3.0,0.0,0.0,1.0,1.0,...,0,1,0,0,1.342171,1.674266,1.353033,0.310371,1.846679,4.283248
9964,56.449611,9.404525,9500.0,1550.0,130.0,4.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0.466744,33.988380,0.126922,0.030089,0.458502,0.100810
9965,55.307771,11.543354,7150.0,500.0,87.0,3.0,0.0,0.0,0.0,0.0,...,0,1,0,0,12.496495,10.451021,2.480094,8.383154,15.404188,16.237640


In [23]:
# Split the data into a 70% training set and a 30% testing set
train_data, test_data = train_test_split(data1, test_size=0.3, random_state=69)

data = train_data

#Display the data used for training
data

,Latitude,Longitude,Monthly rent,Aconto,Square meters,Rooms,Furnished,Shareable,Pets allowed,Elevator,...,Floor_-1 to 0,Floor_1-3,Floor_3-8,Floor_9 or above,Distance to Transport Station (km),Distance to Beach (km),Distance to School (km),Distance to Restaurant (km),Distance to Hospital (km),Distance to Mall (km)
4190,56.449348,9.378721,5650.0,950.0,79.0,3.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1.239100,34.379012,0.789074,0.619608,1.639672,1.621548
2893,57.054318,9.897610,6400.0,750.0,72.0,2.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0.675900,45.481163,0.425179,0.367922,1.114816,1.368703
2053,55.503723,9.750999,9800.0,0.0,111.0,3.0,0.0,1.0,1.0,0.0,...,0,1,0,0,1.051814,15.169527,0.730248,0.916473,1.116064,8.045266
9238,56.449645,9.407568,5300.0,1100.0,57.0,2.0,0.0,0.0,1.0,0.0,...,0,1,0,0,0.629770,33.947298,0.303447,0.041116,0.336722,0.187833
1075,55.707032,12.574998,17800.0,1000.0,90.0,3.0,0.0,0.0,1.0,1.0,...,0,1,0,0,0.281259,2.933244,0.319177,0.162239,1.115878,2.111714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4374,55.584800,12.320129,11000.0,1000.0,78.0,3.0,0.0,0.0,1.0,0.0,...,0,1,0,0,1.713105,2.465081,0.853753,0.563838,4.611246,1.570729
8631,55.567355,9.756380,6995.0,750.0,107.0,4.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.988413,8.113230,0.389214,0.120704,7.956210,0.973902
9818,55.567406,9.762487,3938.0,700.0,40.0,2.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1.373363,8.151401,0.447950,0.192810,8.025436,0.941141
4041,55.483934,9.472074,6000.0,1000.0,64.0,2.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.945424,3.461496,0.445250,0.259864,1.995298,3.371061


# Without Geospatial Data

In [24]:
# List of variables to remove iteratively
variables_to_remove = ['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0', 'Distance to Transport Station (km)', 'Distance to Beach (km)', 'Distance to School (km)', 'Distance to Restaurant (km)', 'Distance to Hospital (km)', 'Distance to Mall (km)']

# Prepare the features (X) and the label (y)
X = data.drop(variables_to_remove, axis=1)
y = data['Monthly rent']  # Labels (the 'Monthly rent' column)

# Add a constant to the features (intercept term)
X = sm.add_constant(X)

# Initialize a list to keep track of removed variables
removed_variables = []

while True:
    # Create an OLS model
    model = sm.OLS(y, X)

    # Fit the OLS model with heteroscedasticity robust errors
    results = model.fit(cov_type='HC3')

    # Find the variable with the highest p-value (least significant)
    max_p_value = results.pvalues.idxmax()
    
    # Check if the highest p-value is above a certain threshold (e.g., 0.05)
    if results.pvalues[max_p_value] > 0.10:
        print(f"Removing '{max_p_value}' (p-value: {results.pvalues[max_p_value]:.4f})")
        X = X.drop(max_p_value, axis=1)
        removed_variables.append(max_p_value)
    else:
        break

# Print the summary of the final regression model
print(results.summary())

# Get the Predicted Values
y_pred = results.predict(X)

# Calculate the Residuals
residuals = y - y_pred

# Calculate the RMSE
RMSE_train1 = np.sqrt((residuals ** 2).mean())
print('Train Root Mean Squared Error (RMSE):', RMSE_train1)

# Print the list of removed variables
print("Removed variables:", removed_variables)


Removing 'Parking' (p-value: 0.8740)
Removing 'Rooms' (p-value: 0.4017)
Removing 'Floor_1-3' (p-value: 0.3131)
Removing 'Senior frendly' (p-value: 0.2488)
                            OLS Regression Results                            
Dep. Variable:           Monthly rent   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.755
Method:                 Least Squares   F-statistic:                     540.1
Date:                Fri, 18 Aug 2023   Prob (F-statistic):               0.00
Time:                        13:42:14   Log-Likelihood:                -62831.
No. Observations:                6976   AIC:                         1.257e+05
Df Residuals:                    6957   BIC:                         1.258e+05
Df Model:                          18                                         
Covariance Type:                  HC3                                         
                           coef    std err          z  

## Test the Model Without Geospatial Data on the Test Data Set

In [25]:
removed = variables_to_remove + removed_variables
# Prepare the features in the test data for the first model
X_test1 = test_data.drop(removed, axis=1)
X_test1 = sm.add_constant(X_test1)  # Add constant

# Make sure the test data columns match the model's training data columns
X_test1 = X_test1[X.columns]

# True 'Monthly rent' values in the test data
y_test = test_data['Monthly rent']

# Get the Predicted Values on the test data
y_pred_test1 = results.predict(X_test1)

# Calculate the Residuals for the test data
residuals_test1 = y_test - y_pred_test1

# Calculate the RMSE for the test data
RMSE_test1 = np.sqrt((residuals_test1 ** 2).mean())
print('Test Root Mean Squared Error (RMSE) for First Model:', RMSE_test1)

Test Root Mean Squared Error (RMSE) for First Model: 2006.2448110982825


# With Geospatial Data

In [26]:
# Prepare the features (X) and the label (y)
variables_to_remove2 = ['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0']

X = data.drop(variables_to_remove2, axis=1)
y = data['Monthly rent']

# Add a constant to the features (intercept term)
X = sm.add_constant(X)

# Initialize a list to keep track of removed variables
removed_variables2 = []

while True:
    # Create an OLS model
    model = sm.OLS(y, X)

    # Fit the OLS model with heteroscedasticity robust errors
    results = model.fit(cov_type='HC3')

    # Find the variable with the highest p-value (least significant)
    max_p_value = results.pvalues.idxmax()

    # Check if the highest p-value is above a certain threshold (e.g., 0.10)
    if results.pvalues[max_p_value] > 0.10:
        print(f"Removing '{max_p_value}' (p-value: {results.pvalues[max_p_value]:.4f})")
        X = X.drop(max_p_value, axis=1)
        removed_variables2.append(max_p_value)
    else:
        break

# Print the summary of the final regression model
print(results.summary())

# Get the Predicted Values
y_pred = results.predict(X)

# Calculate the Residuals
residuals = y - y_pred

# Calculate the RMSE
RMSE_train2 = np.sqrt((residuals ** 2).mean())
print('Train Root Mean Squared Error (RMSE) for Second Model:', RMSE_train2)

# Print the list of removed variables
print("Removed variables:", removed_variables2)


Removing 'Rooms' (p-value: 0.5141)
Removing 'Senior frendly' (p-value: 0.4157)
Removing 'Distance to School (km)' (p-value: 0.4073)
Removing 'Floor_1-3' (p-value: 0.2281)
                            OLS Regression Results                            
Dep. Variable:           Monthly rent   R-squared:                       0.782
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     467.4
Date:                Fri, 18 Aug 2023   Prob (F-statistic):               0.00
Time:                        13:42:15   Log-Likelihood:                -62427.
No. Observations:                6976   AIC:                         1.249e+05
Df Residuals:                    6951   BIC:                         1.251e+05
Df Model:                          24                                         
Covariance Type:                  HC3                                         
                                       

## Test the Model With Geospatial Data on the Test Data Set

In [27]:
removed2 = variables_to_remove2 + removed_variables2

# Prepare the features in the test data for the second model
X_test2 = test_data.drop(removed2, axis=1)
X_test2 = sm.add_constant(X_test2)  # Add constant

# Make sure the test data columns match the model's training data columns
X_test2 = X_test2[X.columns]

# True 'Monthly rent' values in the test data
y_test = test_data['Monthly rent']

# Get the Predicted Values on the test data
y_pred_test2 = results.predict(X_test2)

# Calculate the Residuals for the test data
residuals_test2 = y_test - y_pred_test2

# Calculate the RMSE for the test data
RMSE_test2 = np.sqrt((residuals_test2 ** 2).mean())
print('Test Root Mean Squared Error (RMSE) for Second Model:', RMSE_test2)

Test Root Mean Squared Error (RMSE) for Second Model: 1908.960570263923


## Estimation and Test of Model without Geospatial Data and Interaction Terms

In [28]:
# Define the floor categories you want to create interaction terms for
floor_categories = ['Floor_1-3', 'Floor_3-8', 'Floor_9 or above']

# Define the postal code ranges you want to create interaction terms for
postal_code_ranges = ['PostalCode_2000-2999', 'PostalCode_3000-3999', 'PostalCode_4000-4999',
                      'PostalCode_5000-5999', 'PostalCode_6000-6999', 'PostalCode_7000-7999',
                      'PostalCode_8000-8999', 'PostalCode_9000-9999']

# Create interaction terms between floor categories and Elevator
for floor_cat in floor_categories:
    interaction_col_name = f'{floor_cat}_x_Elevator'
    train_data[interaction_col_name] = train_data[floor_cat] * train_data['Elevator']
    test_data[interaction_col_name] = test_data[floor_cat] * test_data['Elevator']

# Create interaction terms only between postal codes and square meters
for postal_col in postal_code_ranges:
    interaction_col_name = f'{postal_col}_x_Square meters'
    train_data[interaction_col_name] = train_data[postal_col] * train_data['Square meters']
    test_data[interaction_col_name] = test_data[postal_col] * test_data['Square meters']

# Prepare the features (X_train) and the label (y_train) using training data
X_train = train_data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0', 'Distance to Transport Station (km)', 'Distance to Beach (km)', 'Distance to School (km)', 'Distance to Restaurant (km)', 'Distance to Hospital (km)', 'Distance to Mall (km)'], axis=1)
y_train = train_data['Monthly rent']

# Initialize a list to keep track of removed variables
removed_variables3 = []

while True:
    # Add a constant to the features (intercept term)
    X_train = sm.add_constant(X_train)

    # Create an OLS model
    model = sm.OLS(y_train, X_train)

    # Fit the OLS model with heteroscedasticity robust errors
    results = model.fit(cov_type='HC3')

    # Find the variable with the highest p-value (least significant)
    max_p_value = results.pvalues.idxmax()

    # Check if the highest p-value is above a certain threshold (e.g., 0.10)
    if results.pvalues[max_p_value] > 0.10:
        print(f"Removing '{max_p_value}' (p-value: {results.pvalues[max_p_value]:.4f})")
        X_train = X_train.drop(max_p_value, axis=1)
        removed_variables3.append(max_p_value)
    else:
        break

# Print the summary of the final regression model
print(results.summary())

# Use the model to make predictions on the training set
y_pred_train = results.predict(X_train)

# Calculate the Residuals for the training data
residuals_train = y_train - y_pred_train

# Calculate the RMSE for the training data
RMSE_train3 = np.sqrt((residuals_train ** 2).mean())
print('Train Root Mean Squared Error (RMSE):', RMSE_train3)

# Print the list of removed variables
print("Removed variables:", removed_variables3)

# Prepare the features (X_test) and the label (y_test) using testing data
X_test = test_data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0', 'Distance to Transport Station (km)', 'Distance to Beach (km)', 'Distance to School (km)', 'Distance to Restaurant (km)', 'Distance to Hospital (km)', 'Distance to Mall (km)'] + removed_variables3, axis=1)
y_test = test_data['Monthly rent']

# Add a constant to the features (intercept term for test data)
X_test = sm.add_constant(X_test)

# Use the model to make predictions on the test set
y_pred_test = results.predict(X_test)

# Calculate the Residuals for the test data
residuals_test = y_test - y_pred_test

# Calculate the RMSE for the test data
RMSE_test3 = np.sqrt((residuals_test ** 2).mean())
print('Test Root Mean Squared Error (RMSE):', RMSE_test3)


Removing 'Rooms' (p-value: 0.7457)
Removing 'PostalCode_4000-4999' (p-value: 0.7478)
Removing 'Floor_3-8_x_Elevator' (p-value: 0.5645)
Removing 'Floor_1-3_x_Elevator' (p-value: 0.5387)
Removing 'Floor_1-3' (p-value: 0.6096)
Removing 'Parking' (p-value: 0.2797)
Removing 'Senior frendly' (p-value: 0.2141)
Removing 'PostalCode_2000-2999_x_Square meters' (p-value: 0.1432)
Removing 'PostalCode_3000-3999' (p-value: 0.6100)
                            OLS Regression Results                            
Dep. Variable:           Monthly rent   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.788
Method:                 Least Squares   F-statistic:                     680.2
Date:                Fri, 18 Aug 2023   Prob (F-statistic):               0.00
Time:                        13:42:15   Log-Likelihood:                -62319.
No. Observations:                6976   AIC:                         1.247e+05
Df Residuals:             

## Estimation and Test of Model with Geospatial Data and Interaction Terms

In [29]:
# Define the floor categories you want to create interaction terms for
floor_categories = ['Floor_1-3', 'Floor_3-8', 'Floor_9 or above']

# Define the postal code ranges you want to create interaction terms for
postal_code_ranges = ['PostalCode_2000-2999', 'PostalCode_3000-3999', 'PostalCode_4000-4999',
                      'PostalCode_5000-5999', 'PostalCode_6000-6999', 'PostalCode_7000-7999',
                      'PostalCode_8000-8999', 'PostalCode_9000-9999']

# Define the distance variables you want to create interaction terms for
distance_variables = ['Distance to Transport Station (km)', 'Distance to Beach (km)',
                      'Distance to School (km)', 'Distance to Restaurant (km)',
                      'Distance to Hospital (km)', 'Distance to Mall (km)']

# Create interaction terms between floor categories and Elevator
for floor_cat in floor_categories:
    interaction_col_name = f'{floor_cat}_x_Elevator'
    train_data[interaction_col_name] = train_data[floor_cat] * train_data['Elevator']
    test_data[interaction_col_name] = test_data[floor_cat] * test_data['Elevator']

# Create interaction terms between postal code ranges and distance variables
for postal_col in postal_code_ranges:
    for dist_var in distance_variables:
        interaction_col_name = f'{postal_col}_x_{dist_var}'
        train_data[interaction_col_name] = train_data[postal_col] * train_data[dist_var]
        test_data[interaction_col_name] = test_data[postal_col] * test_data[dist_var]

# Create interaction terms only between postal codes and square meters
for postal_col in postal_code_ranges:
    interaction_col_name = f'{postal_col}_x_Square meters'
    train_data[interaction_col_name] = train_data[postal_col] * train_data['Square meters']
    test_data[interaction_col_name] = test_data[postal_col] * test_data['Square meters']

# Prepare the features (X_train) and the label (y_train) using training data
X_train = train_data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0'], axis=1)
y_train = train_data['Monthly rent']

# Initialize a list to keep track of removed variables
removed_variables4 = []

while True:
    # Add a constant to the features (intercept term)
    X_train = sm.add_constant(X_train)

    # Create an OLS model
    model = sm.OLS(y_train, X_train)

    # Fit the OLS model with heteroscedasticity robust errors
    results = model.fit(cov_type='HC3')

    # Find the variable with the highest p-value (least significant)
    max_p_value = results.pvalues.idxmax()

    # Check if the highest p-value is above a certain threshold (e.g., 0.10)
    if results.pvalues[max_p_value] > 0.10:
        print(f"Removing '{max_p_value}' (p-value: {results.pvalues[max_p_value]:.4f})")
        X_train = X_train.drop(max_p_value, axis=1)
        removed_variables4.append(max_p_value)
    else:
        break

# Print the summary of the final regression model
print(results.summary())

# Use the model to make predictions on the training set
y_pred_train = results.predict(X_train)

# Calculate the Residuals for the training data
residuals_train = y_train - y_pred_train

# Calculate the RMSE for the training data
RMSE_train4 = np.sqrt((residuals_train ** 2).mean())
print('Train Root Mean Squared Error (RMSE):', RMSE_train4)

# Print the list of removed variables
print("Removed variables:", removed_variables4)

# Prepare the features (X_test) and the label (y_test) using testing data
X_test = test_data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0'] + removed_variables4, axis=1)
y_test = test_data['Monthly rent']

# Add a constant to the features (intercept term for test data)
X_test = sm.add_constant(X_test)

# Use the model to make predictions on the test set
y_pred_test = results.predict(X_test)

# Calculate the Residuals for the test data
residuals_test = y_test - y_pred_test

# Calculate the RMSE for the test data
RMSE_test4 = np.sqrt((residuals_test ** 2).mean())
print('Test Root Mean Squared Error (RMSE):', RMSE_test4)


Removing 'PostalCode_2000-2999_x_Distance to Transport Station (km)' (p-value: 0.9997)
Removing 'Senior frendly' (p-value: 0.8498)
Removing 'Rooms' (p-value: 0.6914)
Removing 'PostalCode_4000-4999' (p-value: 0.5581)
Removing 'Floor_9 or above_x_Elevator' (p-value: 0.4816)
Removing 'PostalCode_4000-4999_x_Distance to School (km)' (p-value: 0.3736)
Removing 'PostalCode_5000-5999_x_Distance to School (km)' (p-value: 0.6300)
Removing 'PostalCode_8000-8999_x_Distance to School (km)' (p-value: 0.5564)
Removing 'PostalCode_2000-2999_x_Distance to Hospital (km)' (p-value: 0.4372)
Removing 'Aconto' (p-value: 0.3280)
Removing 'PostalCode_2000-2999_x_Distance to Mall (km)' (p-value: 0.3226)
Removing 'PostalCode_2000-2999_x_Distance to School (km)' (p-value: 0.2237)
Removing 'Distance to Mall (km)' (p-value: 0.2598)
Removing 'PostalCode_2000-2999_x_Square meters' (p-value: 0.1308)
Removing 'PostalCode_2000-2999_x_Distance to Beach (km)' (p-value: 0.2152)
Removing 'PostalCode_3000-3999' (p-value: 0

# Test/Train Summary

In [30]:
# Create a dictionary with the RMSE values
rmse_data = {
    'Data Split': ['Train', 'Train', 'Train', 'Train', 'Test', 'Test', 'Test', 'Test'],
    'Model': ['Without Geospatial Data', 'With Geospatial Data', 'Without Geospatial Data but with Interaction Terms', 'With Geospatial Data and Interaction Terms', 'Without Geospatial Data', 'With Geospatial Data', 'Without Geospatial Data but with Interaction Terms', 'With Geospatial Data and Interaction Terms'],
    'RMSE': [RMSE_train1, RMSE_train2, RMSE_train3, RMSE_train4, RMSE_test1, RMSE_test2, RMSE_test3, RMSE_test4]
}

# Convert the dictionary to a Pandas DataFrame
rmse_df = pd.DataFrame(rmse_data)

# Print the DataFrame as a table
print(rmse_df)

,Data Split,Model,RMSE
0,Train,Without Geospatial Data,1973.959556
1,Train,With Geospatial Data,1862.913050
2,Train,Without Geospatial Data but with Interaction T...,1834.246807
3,Train,With Geospatial Data and Interaction Terms,1551.790768
4,Test,Without Geospatial Data,2006.244811
5,Test,With Geospatial Data,1908.960570
6,Test,Without Geospatial Data but with Interaction T...,1838.426823
7,Test,With Geospatial Data and Interaction Terms,1580.118722
